*Give credit for code foundation here*

### Grab vocab list from Github
This can be changed at a later time. All that matter is that the vocab names are directly from QuickDraw and that the size of the output layer reflects the number of items in the text file.

In [2]:
!wget https://raw.githubusercontent.com/Capstone-Projects-2023-Fall/project-smartspeech/fix-suggested-tiles/backend/model/quickdraw-vocab.txt

--2023-12-04 02:19:25--  https://raw.githubusercontent.com/Capstone-Projects-2023-Fall/project-smartspeech/fix-suggested-tiles/backend/model/quickdraw-vocab.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 296 [text/plain]
Saving to: ‘quickdraw-vocab.txt’

quickdraw-vocab.txt 100%[===================>]     296  --.-KB/s    in 0s      

2023-12-04 02:19:25 (6.17 MB/s) - ‘quickdraw-vocab.txt’ saved [296/296]



### Store class names into a list
Remove newline characters and replace all spaces with underscores so that the data is equally represented.

In [3]:
f = open('quickdraw-vocab.txt', 'r')
classes = f.readlines()
f.close()

In [4]:
classes = [c.replace('\n', '').replace(' ', '_') for c in classes]

### Download data from Google


In [5]:
!mkdir data

In [6]:
import urllib.request
def download():
  base = 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/'
  for c in classes:
    cls_url = c.replace('_', '%20')
    path = base+cls_url+'.npy'
    print(path)
    urllib.request.urlretrieve(path, 'data/'+c+'.npy')

In [7]:
download()

https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/blueberry.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/car.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/carrot.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/chair.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/church.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/circle.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/diamond.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/door.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/finger.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/flower.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/foot.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/fork.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/ham

### Imports

In [8]:
import os
import glob
import numpy as np
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf

### Load data

In [25]:
def load_data(root, vfold_ratio=0.2, max_items_per_class= 10000):
    all_files = glob.glob(os.path.join(root, '*.npy'))

    #initialize variables
    x = np.empty([0, 784])
    y = np.empty([0])
    class_names = []

    #load each data file
    for idx, file in enumerate(all_files):
        data = np.load(file)
        data = data[0: max_items_per_class, :]
        labels = np.full(data.shape[0], idx)

        x = np.concatenate((x, data), axis=0)
        y = np.append(y, labels)

        class_name, ext = os.path.splitext(os.path.basename(file))
        class_names.append(class_name)

    data = None
    labels = None

    #randomize the dataset
    permutation = np.random.permutation(y.shape[0])
    x = x[permutation, :]
    y = y[permutation]

    #separate into training and testing
    vfold_size = int(x.shape[0]/100*(vfold_ratio*100))

    x_test = x[0:vfold_size, :]
    y_test = y[0:vfold_size]

    x_train = x[vfold_size:x.shape[0], :]
    y_train = y[vfold_size:y.shape[0]]
    return x_train, y_train, x_test, y_test, class_names

In [26]:
x_train, y_train, x_test, y_test, class_names = load_data('data')
num_classes = len(class_names)
image_size = 28

In [27]:
print(len(x_train))

320000


### Preprocess data

In [28]:
# Reshape and normalize
x_train = x_train.reshape(x_train.shape[0], image_size, image_size, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], image_size, image_size, 1).astype('float32')

x_train /= 255.0
x_test /= 255.0

# Convert class vectors to class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

### Define model

In [29]:
# Define model
model = keras.Sequential()
model.add(layers.Convolution2D(16, (3, 3),
                        padding='same',
                        input_shape=x_train.shape[1:], activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Convolution2D(32, (3, 3), padding='same', activation= 'relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Convolution2D(64, (3, 3), padding='same', activation= 'relu'))
model.add(layers.MaxPooling2D(pool_size =(2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='tanh'))
model.add(layers.Dense(40, activation='softmax'))
# Train model
adam = tf.optimizers.Adam()
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['top_k_categorical_accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 28, 28, 16)        160       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 14, 14, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 14, 14, 32)        4640      
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 7, 7, 32)          0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 7, 7, 64)          18496     
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 3, 3, 64)         

### Train model

In [30]:
history = model.fit(x = x_train, y = y_train, validation_split=0.1, batch_size = 256, verbose=2, epochs=5)

Epoch 1/5
1125/1125 - 8s - loss: 1.0514 - top_k_categorical_accuracy: 0.9047 - val_loss: 0.7067 - val_top_k_categorical_accuracy: 0.9492 - 8s/epoch - 7ms/step
Epoch 2/5
1125/1125 - 5s - loss: 0.6423 - top_k_categorical_accuracy: 0.9532 - val_loss: 0.5993 - val_top_k_categorical_accuracy: 0.9571 - 5s/epoch - 5ms/step
Epoch 3/5
1125/1125 - 6s - loss: 0.5646 - top_k_categorical_accuracy: 0.9592 - val_loss: 0.5513 - val_top_k_categorical_accuracy: 0.9597 - 6s/epoch - 5ms/step
Epoch 4/5
1125/1125 - 5s - loss: 0.5181 - top_k_categorical_accuracy: 0.9630 - val_loss: 0.5783 - val_top_k_categorical_accuracy: 0.9589 - 5s/epoch - 5ms/step
Epoch 5/5
1125/1125 - 6s - loss: 0.4859 - top_k_categorical_accuracy: 0.9657 - val_loss: 0.5157 - val_top_k_categorical_accuracy: 0.9635 - 6s/epoch - 5ms/step


### Test model

In [31]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test accuarcy: {:0.2f}%'.format(score[1] * 100))

Test accuarcy: 96.31%


### Store classes

In [32]:
with open('class_names.txt', 'w') as file_handler:
    for item in class_names:
        file_handler.write("{}\n".format(item))

### Export weights and relevant files

In [19]:
!pip install tensorflowjs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 78.3 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.14.0
    Uninstalling tensorflow-estimator-2.14.0:
      Successfully uninstalled tensorflow-estimator-2.14.0
  Attempting uninstall: keras
    Found existing installation: keras 2.14.0
    Uninstalling keras-2.14.0:
      Successfully uninstalled keras-2.14.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.14.1
    Uninstalling tensorboard-2.14.1:
      Successfully uninstalle

In [33]:
model.save('keras.h5')

In [34]:
!mkdir model
!tensorflowjs_converter --input_format keras keras.h5 model/

2023-12-04 03:07:00.043429: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-04 03:07:00.043521: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-04 03:07:00.177314: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-04 03:07:02.318086: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [35]:
!cp class_names.txt model/class_names.txt

In [37]:
!zip -r model.zip model

  adding: model/ (stored 0%)
  adding: model/group1-shard1of1.bin (deflated 7%)
  adding: model/class_names.txt (deflated 35%)
  adding: model/model.json (deflated 82%)


In [ ]:
from google.colab import files
files.download('model.zip')